# 1. Connect to SQL Database

- It would be bad practice to store your database password into a Python script or Jupyter Notebook
- Instead, we store the password in a separate `.env` file. 
- This is a text file that contains environment variables in the form of name=value pairs. 
- You can then load these variables into environment variables using the dotenv package.
- Such a .env file should never be shared with others or checked into version control (will be covered in our lecture on git and Github)

In [1]:
from dotenv import load_dotenv                        # Load passwords etc from .env file 
load_dotenv('.env')                                   # Absolute or relative path to the .env file 

True

- The package SQLAlchemy allows you to connect to different variants of SQL databases (e.g. MySQL, PostgreSQL, SQLite, ...).
- To connect to a MySQL database, you need to specify both the database dialect (mysql) and choose an appropriate driver (pymysql or mysqlconnector).

In [2]:
from sqlalchemy import create_engine                  # Connection to database
import os                                             # Needed to access environment variables

In [3]:
DIALECT = 'mysql'
DRIVER = 'pymysql'
USER = os.getenv('MYSQL_USER')
PASSWORD = os.getenv('MYSQL_PASSWORD')
HOST = 'localhost'
PORT = '3306'
DB = 'music'

connection_string = f"{DIALECT}+{DRIVER}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}"
engine = create_engine(connection_string)

# 2. Read from SQL Database to Pandas DataFrame

## 2.1 Inspecting the MySQL database

First, we inspect the available databases in our MySQL Server, and specifically the available tables in our "music" database.

In [4]:
import pandas as pd
from sqlalchemy import inspect

In [5]:
inspector = inspect(engine)

print('Available Databases:')
inspector.get_schema_names()

Available Databases:


['information_schema',
 'music',
 'mysql',
 'performance_schema',
 'sales',
 'sys',
 'test']

In [6]:
inspector.get_table_names()

['artists', 'avg_streams_by_region', 'charts', 'tracks', 'tracks_artists']

## 2.2 Read full table

We can read entire tables into a Pandas Dataframe

In [7]:
tracks = pd.read_sql('tracks_artists', con=engine)
tracks.head()

,track_id,artist_id
0,1xK1Gg9SxG8fy2Ya373oqb,7iK8PXO48WeuP03g8YR51W
1,1xK1Gg9SxG8fy2Ya373oqb,2LmcxBak1alK1bf7d1beTr
2,1uXJYnCsXy2v50y0fDEEiV,7gjqZ8coFZimZDtdk04WP1
3,1uXJYnCsXy2v50y0fDEEiV,1MRiIeZbc0cRuxOafDUCtH
4,1uXJYnCsXy2v50y0fDEEiV,3hBQ4zniNdQf1cqqo6hzuW


## 2.3 SQL Select Query

We can send arbitrary SQL queries to read data into a Pandas DataFrame

### Select all columns

In [8]:
query = "select * from charts limit 3"
pd.read_sql(query, con=engine)

,id,region,freq,date,track_id,rank,streams,peak_rank,weeks_on_chart
0,1,de,weekly,2021-06-03,4ZtFanR9U6ndgddUvNcjcG,1,3704439,1,3.0
1,1,de,weekly,2021-06-03,6DiaHETMcKdX7FwuGXwp1O,2,3582697,2,1.0
2,1,de,weekly,2021-06-03,4txsSBdT4yVJtUj41YaQEB,3,2945134,1,3.0


### Where conditions

In [9]:
query = "select * from charts where streams >  17000000"
pd.read_sql(query, con=engine)

,id,region,freq,date,track_id,rank,streams,peak_rank,weeks_on_chart
0,66,us,weekly,2021-06-03,4ZtFanR9U6ndgddUvNcjcG,1,24777068,1,3.0
1,67,us,weekly,2021-06-10,4ZtFanR9U6ndgddUvNcjcG,1,20688190,1,4.0
2,68,us,weekly,2021-06-17,4ZtFanR9U6ndgddUvNcjcG,1,17136875,1,5.0


### Aggregations

In [10]:
query = "select count(*), avg(streams) from charts "
pd.read_sql(query, con=engine)

,count(*),avg(streams)
0,15598,1.030511e+06


### Grouped aggregations

In [11]:
query = "select region, avg(streams) from charts group by region order by avg(streams) desc"
pd.read_sql(query, con=engine)

,region,avg(streams)
0,us,2.606887e+06
1,de,8.139232e+05
2,es,7.866070e+05
3,it,7.221330e+05
4,gb,6.968614e+05
5,fr,5.564647e+05


### Joining tables

In [12]:
query = """select * from charts
           left join tracks
           on charts.track_id = tracks.id"""
tracks = pd.read_sql(query, con=engine)

In [13]:
tracks.sort_values(by = 'danceability', ascending = True, inplace= True)
tracks.head()

,id,region,freq,date,track_id,rank,streams,peak_rank,weeks_on_chart,id,...,mode,tempo,explicit,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence
9708,49,gb,weekly,2021-08-05,5jhBwnqzNNrENXnYrAdoCe,111,489232,111,1.0,5jhBwnqzNNrENXnYrAdoCe,...,minor,65.136,0,0.275,0.155,0.0674,0.928,0.176,0.0888,0.0588
14865,75,us,weekly,2021-08-05,5jhBwnqzNNrENXnYrAdoCe,68,2426489,68,1.0,5jhBwnqzNNrENXnYrAdoCe,...,minor,65.136,0,0.275,0.155,0.0674,0.928,0.176,0.0888,0.0588
11574,58,it,weekly,2021-07-08,7JlPKH9G2KLUzJlfFFAxEO,177,290357,57,13.0,7JlPKH9G2KLUzJlfFFAxEO,...,minor,185.135,1,0.296,0.486,0.0327,0.311,0.000,0.0980,0.1590
11348,57,it,weekly,2021-07-01,7JlPKH9G2KLUzJlfFFAxEO,151,348419,57,12.0,7JlPKH9G2KLUzJlfFFAxEO,...,minor,185.135,1,0.296,0.486,0.0327,0.311,0.000,0.0980,0.1590
11988,60,it,weekly,2021-07-22,7JlPKH9G2KLUzJlfFFAxEO,191,272728,57,15.0,7JlPKH9G2KLUzJlfFFAxEO,...,minor,185.135,1,0.296,0.486,0.0327,0.311,0.000,0.0980,0.1590


In [14]:
query = "select * from tracks order by danceability asc limit 5"
pd.read_sql(query, con=engine)


,id,name,duration,mode,tempo,explicit,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence
0,5jhBwnqzNNrENXnYrAdoCe,Everybody Dies,206.62,minor,65.136,0,0.275,0.155,0.0674,0.9280,0.176000,0.0888,0.0588
1,7JlPKH9G2KLUzJlfFFAxEO,MARILÙ,250.83,minor,185.135,1,0.296,0.486,0.0327,0.3110,0.000000,0.0980,0.1590
2,2wOnrN7fkzV4NsLqym2C6O,Already Won (feat. Lil Durk),212.06,minor,75.774,1,0.321,0.525,0.1310,0.0668,0.000000,0.1370,0.1520
3,3NM41PVVUr0ceootKAtkAj,As the World Caves In,218.75,mayor,123.969,0,0.327,0.449,0.0340,0.3690,0.004440,0.2200,0.1790
4,0UvCh63URrLFcPkKt99hHd,Don't Look Back in Anger - Remastered,289.56,mayor,162.937,0,0.327,0.938,0.0645,0.0708,0.000004,0.1480,0.3260


In [15]:
tracks.columns

Index(['id', 'region', 'freq', 'date', 'track_id', 'rank', 'streams',
       'peak_rank', 'weeks_on_chart', 'id', 'name', 'duration', 'mode',
       'tempo', 'explicit', 'danceability', 'energy', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence'],
      dtype='object')

In [16]:
tracks[['region', 'date', 'streams']].groupby(['region', 'date']).agg({'streams': 'sum'}).reset_index().head(10)

,region,date,streams
0,de,2021-06-03,174951591
1,de,2021-06-10,160331326
2,de,2021-06-17,163419945
3,de,2021-06-24,155525359
4,de,2021-07-01,165842447
5,de,2021-07-08,175064981
6,de,2021-07-15,158801356
7,de,2021-07-22,168857525
8,de,2021-07-29,161810298
9,de,2021-08-05,155194440


In [17]:
query = "select region, date, sum(streams) from charts group by region, date"
pd.read_sql(query, con=engine)

,region,date,sum(streams)
0,de,2021-06-03,174951591.0
1,de,2021-06-10,160331326.0
2,de,2021-06-17,163419945.0
3,de,2021-06-24,155525359.0
4,de,2021-07-01,165842447.0
...,...,...,...
73,us,2021-07-29,503997422.0
74,us,2021-08-05,518532015.0
75,us,2021-08-12,490523006.0
76,us,2021-08-19,485031696.0


In [18]:
# Read in the entire tracks and charts table
tracks = pd.read_sql('tracks', con=engine)
charts = pd.read_sql('charts', con=engine)
# Pandas data exploration
df = pd.merge(left=charts,
right=tracks.rename(columns={'id':'track_id'}),
how='left',
on='track_id')
df_germany = df.query("region=='de'")
df_germany.groupby(['track_id','name']).streams.count().sort_values()

track_id                name                          
7nvaEKdCDWbnQK7uJsCS5y  Impossible (feat. John Martin)     1
042Sl6Mn83JHyLEqdK7uI0  Your Power                         1
7eXnaPqRon4nlnRRdaNubB  MAHALLE                            1
7dJbURdTcF4y5J2sYe3000  Melodie                            1
7bcy34fBT2ap1L4bfPsl9q  I Didn't Change My Number          1
                                                          ..
0Xh6R6XLoTDRBYriQRgkHj  Memories (feat. Kid Cudi)         13
7toCqux0Ln42OttYYyds4k  Frisch                            13
05Mp2UJulSttxQ4E6hQPH3  Ohne mein Team                    13
7ytR5pFWmSjzHJIeQkgog4  ROCKSTAR (feat. Roddy Ricch)      13
02MWAaffLxlfxAUY7c5dvx  Heat Waves                        13
Name: streams, Length: 542, dtype: int64

In [19]:
query = """SELECT charts.track_id, tracks.name, COUNT(charts.streams) AS stream_count
FROM charts
LEFT JOIN tracks
ON charts.track_id = tracks.id
WHERE charts.region = 'de'
GROUP BY charts.track_id, tracks.name
ORDER BY stream_count;
"""
pd.read_sql(query, con=engine)

,track_id,name,stream_count
0,3VqeTFIvhxu3DIe4eZVzGq,Butter,1
1,4FoyKLZg6CYzWDUCGsukHt,SCHIESSEN (feat. Kontra K),1
2,48e9gA75JKjawtBzoyf72B,Tattergreis,1
3,0bTJGFN9BJSdQ0dgNt5I4N,Ja,1
4,1SAGz419Uj4dfJGZQbS7cE,Was ist los,1
...,...,...,...
537,2qa4plPe9puo6TUIP3uKWT,Saufen aufm Spielplatz,13
538,05Mp2UJulSttxQ4E6hQPH3,Ohne mein Team,13
539,60a0Rd6pjrkxjPbaKzXjfq,In the End,13
540,2QjOHCTQ1Jl3zawyYOpxh6,Sweater Weather,13


In [20]:
query = """SELECT charts.*, tracks_artists.artist_id
           FROM charts
           LEFT JOIN tracks_artists
           ON charts.track_id = tracks_artists.track_id"""
tracks = pd.read_sql(query, con=engine)
artists = pd.read_sql('artists', con=engine)

In [ ]:
query = """SELECT charts.*, tracks_artists.artist_id
           FROM charts
           LEFT JOIN tracks_artists
           ON charts.track_id = tracks_artists.track_id"""
tracks = pd.read_sql(query, con=engine)
artists = pd.read_sql('artists', con=engine)
charts = pd.read_sql('charts', con=engine)
tracks_artists = pd.read_sql('tracks_artists', con=engine)

,id,region,freq,date,track_id,rank,streams,peak_rank,weeks_on_chart
0,1,de,weekly,2021-06-03,4ZtFanR9U6ndgddUvNcjcG,1,3704439,1,3.0
1,1,de,weekly,2021-06-03,6DiaHETMcKdX7FwuGXwp1O,2,3582697,2,1.0
2,1,de,weekly,2021-06-03,4txsSBdT4yVJtUj41YaQEB,3,2945134,1,3.0
3,1,de,weekly,2021-06-03,2knDuFWczh1Wv92eBAdPTa,4,2739815,1,6.0
4,1,de,weekly,2021-06-03,3idbCPumlga23rxFA08fZE,5,2664580,5,1.0
...,...,...,...,...,...,...,...,...,...
15593,78,us,weekly,2021-08-26,38GBNKZUhfBkk3oNlWzRYd,196,1450097,13,7.0
15594,78,us,weekly,2021-08-26,7fBv7CLKzipRk6EC6TWHOB,197,1446058,27,41.0
15595,78,us,weekly,2021-08-26,7FIWs0pqAYbP91WWM0vlTQ,198,1445715,2,81.0
15596,78,us,weekly,2021-08-26,07MDkzWARZaLEdKxo6yArG,199,1442195,199,1.0


In [36]:
charts.drop(columns=['id'], inplace=True)

In [40]:
pd.set_option('display.max_columns', None)

In [42]:
charts[charts['track_id']== '4ZtFanR9U6ndgddUvNcjcG']


,region,freq,date,track_id,rank,streams,peak_rank,weeks_on_chart
0,de,weekly,2021-06-03,4ZtFanR9U6ndgddUvNcjcG,1,3704439,1,3.0
200,de,weekly,2021-06-10,4ZtFanR9U6ndgddUvNcjcG,1,3751564,1,4.0
400,de,weekly,2021-06-17,4ZtFanR9U6ndgddUvNcjcG,1,3631925,1,5.0
602,de,weekly,2021-06-24,4ZtFanR9U6ndgddUvNcjcG,3,3361453,1,6.0
802,de,weekly,2021-07-01,4ZtFanR9U6ndgddUvNcjcG,3,3331901,1,7.0
...,...,...,...,...,...,...,...,...
14600,us,weekly,2021-07-29,4ZtFanR9U6ndgddUvNcjcG,3,10599112,1,11.0
14801,us,weekly,2021-08-05,4ZtFanR9U6ndgddUvNcjcG,4,9522059,1,12.0
15001,us,weekly,2021-08-12,4ZtFanR9U6ndgddUvNcjcG,4,8782159,1,13.0
15200,us,weekly,2021-08-19,4ZtFanR9U6ndgddUvNcjcG,3,8456478,1,14.0


In [39]:
charts[charts['track_id']== '1xK1Gg9SxG8fy2Ya373oqb']

,region,freq,date,track_id,rank,streams,peak_rank,weeks_on_chart
2627,es,weekly,2021-06-03,1xK1Gg9SxG8fy2Ya373oqb,28,1233896,1,25.0
2830,es,weekly,2021-06-10,1xK1Gg9SxG8fy2Ya373oqb,31,1176644,1,26.0
3036,es,weekly,2021-06-17,1xK1Gg9SxG8fy2Ya373oqb,37,1109471,1,27.0
3235,es,weekly,2021-06-24,1xK1Gg9SxG8fy2Ya373oqb,36,1063870,1,28.0
3445,es,weekly,2021-07-01,1xK1Gg9SxG8fy2Ya373oqb,46,948921,1,29.0
3648,es,weekly,2021-07-08,1xK1Gg9SxG8fy2Ya373oqb,49,840297,1,30.0
3864,es,weekly,2021-07-15,1xK1Gg9SxG8fy2Ya373oqb,65,712026,1,31.0
4066,es,weekly,2021-07-22,1xK1Gg9SxG8fy2Ya373oqb,67,694394,1,32.0
4272,es,weekly,2021-07-29,1xK1Gg9SxG8fy2Ya373oqb,73,666684,1,33.0
4474,es,weekly,2021-08-05,1xK1Gg9SxG8fy2Ya373oqb,76,636132,1,34.0


In [37]:
tracks = pd.merge(left=tracks_artists,right= charts, how='left', on='track_id')
tracks.head()

,track_id,artist_id,region,freq,date,rank,streams,peak_rank,weeks_on_chart
0,1xK1Gg9SxG8fy2Ya373oqb,7iK8PXO48WeuP03g8YR51W,es,weekly,2021-06-03,28,1233896,1,25.0
1,1xK1Gg9SxG8fy2Ya373oqb,7iK8PXO48WeuP03g8YR51W,es,weekly,2021-06-10,31,1176644,1,26.0
2,1xK1Gg9SxG8fy2Ya373oqb,7iK8PXO48WeuP03g8YR51W,es,weekly,2021-06-17,37,1109471,1,27.0
3,1xK1Gg9SxG8fy2Ya373oqb,7iK8PXO48WeuP03g8YR51W,es,weekly,2021-06-24,36,1063870,1,28.0
4,1xK1Gg9SxG8fy2Ya373oqb,7iK8PXO48WeuP03g8YR51W,es,weekly,2021-07-01,46,948921,1,29.0


In [ ]:
tracks = pd.merge(left=tracks,right= artists, how='left', left_on='artist_id', right_on='id')
tracks.head()

,id_x,region,freq,date,track_id,rank,streams,peak_rank,weeks_on_chart,artist_id,...,soul,house,latin,reggaeton,singer-songwriter,jazz,dance,folk,alternative,country
0,1,de,weekly,2021-06-03,4ZtFanR9U6ndgddUvNcjcG,1,3704439,1,3.0,1McMsnEElThX1knmY4oliG,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,de,weekly,2021-06-03,6DiaHETMcKdX7FwuGXwp1O,2,3582697,2,1.0,2Im8m4STDBosjfmb5hmP80,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,de,weekly,2021-06-03,6DiaHETMcKdX7FwuGXwp1O,2,3582697,2,1.0,1pkDzgHvVqC17qYGKfLPpb,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,de,weekly,2021-06-03,4txsSBdT4yVJtUj41YaQEB,3,2945134,1,3.0,0JBdTCGs111JKKYfLqOEBa,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,de,weekly,2021-06-03,2knDuFWczh1Wv92eBAdPTa,4,2739815,1,6.0,4sVieJlKgpZ2k9ESNS5IdN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tracks[(tracks['region'] == 'de') & (tracks['date'] <= '2021-06-03') & (tracks['date'] >= '2021-06-10') & (tracks.pop == '1')].groupby(['track_id','name'])

,id_x,region,freq,date,track_id,rank,streams,peak_rank,weeks_on_chart,artist_id,...,soul,house,latin,reggaeton,singer-songwriter,jazz,dance,folk,alternative,country


# 3. Write Pandas DataFrame to SQL Database

For this demo, we create some new DataFrame with aggregated data, and then write this data back to the database

In [ ]:
query = """select region, avg(streams) as avg_streams from charts
           left join tracks
           on charts.track_id = tracks.id 
           group by region
           order by avg_streams desc"""
aggregated_data = pd.read_sql(query, con=engine)
aggregated_data

,region,avg_streams
0,us,2.606887e+06
1,de,8.139232e+05
2,es,7.866070e+05
3,it,7.221330e+05
4,gb,6.968614e+05
5,fr,5.564647e+05


In [ ]:
aggregated_data.to_sql(name="avg_streams_by_region", 
                       con=engine, 
                       if_exists='replace',   # fail, replace, append 
                       index=False)

6

In [ ]:
inspect(engine).get_table_names()

['artists', 'avg_streams_by_region', 'charts', 'tracks', 'tracks_artists']

# 4. Execute Arbitrary SQL

So far, we have only used Pandas `read_sql` and `write_sql` functions to interact with our database. **However, we can run arbitrary SQL from our Python session**. 

In the following we:

1. Connect to our MySQL Server
2. We create a new test database
3. We create a new table from a Pandas DataFrame
4. We clean up by dropping the table and the database


In [ ]:
from sqlalchemy import text
import seaborn as sns

### Connect to MySQL Server

Here we do not connect to a specific database, but to the MySQL Server (one level higher in the hierarchy). This will allow us to manage (create, alter, delete ...)  entire databases

In [ ]:
connection_string = f"{DIALECT}+{DRIVER}://{USER}:{PASSWORD}@{HOST}:{PORT}"
server_engine = create_engine(connection_string)

### Create a new database

In [ ]:
with server_engine.connect() as connection:
    connection.execute(text('DROP DATABASE IF EXISTS test'))
    connection.execute(text('CREATE DATABASE test'))

In [ ]:
connection_string = f"{DIALECT}+{DRIVER}://{USER}:{PASSWORD}@{HOST}:{PORT}/test"
test_engine = create_engine(connection_string)

In [ ]:
titanic = sns.load_dataset('titanic')
titanic.to_sql(name='titanic', con=test_engine)

891

### Clean up: drop table and database

In [ ]:
# with server_engine.connect() as connection:
#     connection.execute(text("Drop table if exists test.titanic"))
#     connection.execute(text("Drop database if exists test"))

Close all open connections, if present.

In [ ]:
# engine.dispose()
# server_engine.dispose()
# test_engine.dispose()